<h1>Scraping Fidelity.com</h1>
In this assignment, you will scrape data from fidelity.com. The goal of the exercise is to get the latest sector performance data from the US markets, and to get the total market capitalization for each sector. 

The end result is to write a function: <i>get_us_sector_performance()</i> that will return a list of tuples. Each tuple should correspond to a sector and should contain the following data:
<li>the sector name
<li>the amount the sector has moved
<li>the market capitalization of the sector
<li>the market weight of the sector
<li>a link to the fidelity page for that sector

<p>
The data should be sorted by decreasing order of market weight. I.e., the sector with the highest weight should be in the first tuple, etc.

<h3>Process</h3>
<li>Get a list of sectors and the links to the sector detail pages from the url (see function)
<li>Loop through the list and call the function <i>get_sector_change_and_market_cap(sector_page_link)</i> for each sector
<li>Accumulate the name, the change, the capitalization, the weight and the link for each sector in output_list (see function)
<li>Sort the list by market weight

<b>Notes:</b>
<li>Note that the market weight is a string with a % sign at the back. You will need to get rid of the % and convert the string into a float before you can sort it
<li>Your starting data is the url listed below. You need to extract all data, including links to the sector pages, from the page at this url
<li>To sort a list of tuples by an arbitrary element, use the example at the bottom of this notebook

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def get_us_sector_performance():
    output_list = list()
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"

    response = requests.get(url)
    if not response.status_code == 200:
        return None
    try:
        result_page = BeautifulSoup(response.content,'lxml')
        sector_lst = result_page.select("table.sector-list tbody tr th div")
        for sector in sector_lst:
            sector_name = sector.a.get_text()
            sector_link = "https://eresearch.fidelity.com" + sector.a.get('href')
            sector_info = (sector_name,) + get_sector_change_and_market_cap(sector_link) + (sector_link,)
            output_list.append(sector_info)
    except:
        return None

    output_list.sort(key=lambda k: k[3], reverse = True)
    return output_list

In [3]:
def get_sector_change_and_market_cap(sector_page_link):
    
    response = requests.get(sector_page_link)
    if not response.status_code == 200:
        return None
    try:
        result_page = BeautifulSoup(response.content,'lxml')
        info_lst = result_page.select("table.snapshot-data-tbl tbody tr td")
        sector_change = float(info_lst[0].find('span').get_text()[:-1])
        sector_market_cap = info_lst[1].find('span').get_text()
        sector_market_weight = float(info_lst[2].find('span').get_text()[:-1])
    except:
        return None
    
    return sector_change,sector_market_cap,sector_market_weight

In [4]:
#Test get_sector_change_and_market_cap()
link = "https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25"
get_sector_change_and_market_cap(link)
#Should return (-0.40, $3.58T, 6.80)
#Note that neither the -0.40, nor the 6.80, end with a %sign

(0.62, '$8.82T', 12.32)

In [5]:
#Test get_us_sector_performance()
get_us_sector_performance()


[('Information Technology',
  1.44,
  '$15.52T',
  28.04,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45'),
 ('Health Care',
  0.07,
  '$8.08T',
  13.29,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Consumer Discretionary',
  0.62,
  '$8.82T',
  12.32,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Communication Services',
  1.78,
  '$6.70T',
  11.31,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Financials',
  1.59,
  '$8.79T',
  11.22,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Industrials',
  1.41,
  '$5.72T',
  8.02,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 (

<h3>Sorting</h3>

In [2]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort() #Sorts by the first element of the tuple
x

[('a', 23.2, 'b'), ('c', 17.4, 'f'), ('d', 29.2, 'z'), ('e', 1.74, 'bb')]

In [3]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort(key=lambda k: k[1]) #Sorts by the element at position 1
x

[('e', 1.74, 'bb'), ('c', 17.4, 'f'), ('a', 23.2, 'b'), ('d', 29.2, 'z')]